<a href="https://colab.research.google.com/github/kkorhone/Infinite_Borehole_Field/blob/main/budapest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pygfunction test

This code attempts to find how much energy can be annually extracted from the ground using a 2-by-5 borehole field.

**First, the pygfunction library needs to be installed.**

In [2]:
pip install pygfunction

     |████████████████████████████████| 116 kB 29.5 MB/s 
     |████████████████████████████████| 4.2 MB 62.5 MB/s 
     |████████████████████████████████| 38.1 MB 322 kB/s 
     |████████████████████████████████| 11.2 MB 66.7 MB/s 
     |████████████████████████████████| 895 kB 65.4 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [1]:
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.optimize
import scipy.signal
import pygfunction
import numpy as np

def main(NB):

    monthly_fraction = np.ones(12) / 12

    T_surface = 5.8                         # [degC]
    q_geothermal = 42.9e-3                  # [W/m^2]
    
    k_rock = 2.71                           # [W/(m*K)]
    Cp_rock = 728.0                         # [J/(kg*K)]
    rho_rock = 2731.0                       # [kg/m^3]
    
    R_borehole = 0.085                      # [K/(W/m)]
    
    borehole_length = 200.0                 # [m]
    borehole_radius = 0.115 / 2             # [m]

    num_years = 25                          # [1]

    spf = 3.0                               # [1]

    T_target = -1.0                         # [degC]
    
    a_rock = k_rock / (rho_rock * Cp_rock)  # [m^2/s]
    
    t_max = num_years * 365 * 24 * 3600.0   # [s]

    delta_t = 730 * 3600.0                  # [s]
    
    borehole_geometry = (NB, NB)
    borehole_spacing = (20, 20)
    
    T_initial = T_surface + q_geothermal / k_rock * (0.5 * borehole_length)
    
    ts = borehole_length**2 / (9.0 * a_rock)

    borehole_field = pygfunction.boreholes.rectangle_field(N_1=borehole_geometry[0], N_2=borehole_geometry[1], B_1=borehole_spacing[0], B_2=borehole_spacing[1], H=borehole_length, D=0, r_b=borehole_radius)

    total_borehole_length = borehole_geometry[0] * borehole_geometry[1] * borehole_length

    t = pygfunction.utilities.time_geometric(delta_t, t_max, 50)
    g = pygfunction.gfunction.uniform_temperature(borehole_field, t, a_rock, nSegments=1, disp=False)

    ti = np.arange(delta_t, t_max+delta_t, delta_t)
    gi = scipy.interpolate.interp1d(t, g)(ti)
    
    #plt.figure()
    #plt.plot(np.log(t/ts), g, "b.")
    #plt.plot(np.log(ti/ts), gi, "r-")
    #plt.xlabel("ln(t/ts)")
    #plt.ylabel("g-function")

    def evaluate_mean_fluid_temperatures(annual_heat_load):

        monthly_heat_load = annual_heat_load * monthly_fraction

        heat_rate = np.ravel(np.tile(monthly_heat_load*1_000_000/730.0, (1, num_years)))

        specific_heat_rate = heat_rate / total_borehole_length
        delta_q = np.hstack((-specific_heat_rate[0], np.diff(-specific_heat_rate)))
        
        T_wall = T_initial + scipy.signal.fftconvolve(delta_q, gi/(2.0*np.pi*k_rock), mode="full")[:len(ti)]
        T_fluid = T_wall - R_borehole * specific_heat_rate
        
        return T_fluid
    
    def cost_function(annual_heat_load):

        T_fluid = evaluate_mean_fluid_temperatures(annual_heat_load)

        return np.abs(np.min(T_fluid) - T_target)
    
    annual_heat_load = scipy.optimize.fminbound(cost_function, 1, 100000, xtol=0.001)
    
    T_fluid = evaluate_mean_fluid_temperatures(annual_heat_load)
    
    #plt.figure()
    #plt.plot(ti/(365*24*3600), T_fluid)
    #plt.axhline(T_target, ls="--", color="k")
    #plt.xlabel("Year")
    #plt.ylabel(u"Mean fluid temperature [\xb0C]")
    #plt.title(f"annual_heat_load = {spf/(spf-1)*annual_heat_load:.3f} MWh")

    print(borehole_geometry[0]*borehole_geometry[1], spf/(spf-1)*annual_heat_load)

if __name__ == "__main__":

    for NB in [41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59]:
        main(NB)


1681 33465.24797155367
1764 35080.365231082425
1849 36733.54174398829
1936 38424.77635054577
2025 40154.06835009765
2116 41921.41945276313
2209 43726.82949226787
2304 45570.2967229286
2401 47451.82375298014


KeyboardInterrupt: ignored